---
**From:** 02_Session 3 preparation/01_Exercise_statistics.ipynb
---


# Exercise
Read the file: `product_prices.csv` (separator ';', decimal separator '.') and do the following exercises:

1. Using the `columns` method modify the names of columns to be: `'province', 'product_types', 'currency', 'group_id', 'product_line', 'value', 'date'`<br>
1. Determine the following positional statistics: mean, standard deviation, percentiles: 0, 10%, 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, 100%.<br>Do the exercise **in two versions**: using dedicated functions and `describe`.
1. Do you agree with this way of generating these values? Why?


In [6]:
import pandas as pd
import os
from pathlib import Path


data_path = Path.cwd() / "01_Data" / "product_prices.csv"

df_raw = pd.read_csv(
    data_path,  # path to the file with data (if we want to enter the name: filepath_or_buffer)
    sep=";",  # column separator
    decimal=",",  # sign separating the whole and fractional parts of a number
)

df_raw.head()  # display the first few rows and check if the data actually got loaded (the function itself will be discussed later)

,Name,Goods types,Measurement unit,Group ID,Product types,Value,Date
0,SUBCARPATHIA,NaN,PLN,2,pork ham cooked - per 1kg,21.37,2013-3
1,ŁÓDŹ,NaN,PLN,4,bread - per 1kg,NaN,2018-2
2,KUYAVIA-POMERANIA,NaN,PLN,2,barley groats sausage - per 1kg,3.55,2019-12
3,LOWER SILESIA,NaN,PLN,2,dressed chickens - per 1kg,6.14,2019-2
4,WARMIA-MASURIA,NaN,PLN,2,Italian head cheese - per 1kg,5.63,2002-3


In [9]:
# change types of the df_raw columns
df_typed = df_raw.astype(
    {
        "Name": "string",
        "Goods types": "string",
        "Measurement unit": "string",
        "Group ID": "Int64",
        "Product types": "string",
        "Value": "float",
    }
)

# delete rows with missing values in Value
df_typed = df_typed.dropna(subset=["Value"])

# jednoduché rozdělení (pokud všechny hodnoty mají formát "YYYY-M" nebo "YYYY-MM")
df_typed[["Year", "Month"]] = (
    df_typed["Date"]
    .astype(str)
    .str.split("-", expand=True)
    .iloc[:, :2]
    .apply(pd.to_numeric, errors="coerce")
    .astype("Int64")  # nullable integer dtype
)

# Delete column Date
df_typed = df_typed.drop(columns=["Date"])

# Create new column with quarter information
df_typed["Quarter"] = ((df_typed["Month"] - 1) // 3 + 1).astype("Int64")

# df_typed.head()  # display the first few rows and check the changes

In [11]:
# rename columns
df_renamed = df_typed.rename(
    columns={
        "Name": "province",
        "Goods types": "product_types",
        "Measurement unit": "currency",
        "Group ID": "group_id",
        "Product types": "product_line",
        "Value": "value",
    }
)

In [25]:
# using functions to describe statistics of the column value
print("mean: " + str(df_renamed["value"].mean()))
print("std: " + str(df_renamed["value"].std()))
print()

percentiles = [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]
q = df_renamed["value"].quantile(percentiles)
labels = [f"{int(p*100)}%" for p in percentiles]
table = pd.DataFrame({"value": q.values}, index=labels)
print(table)

mean: 6.615226740294512
std: 34.112858158209036

        value
0%       0.00
10%      0.00
20%      0.00
30%      0.28
40%      1.55
50%      3.09
60%      5.75
70%      9.16
80%     12.73
90%     17.19
100%  3000.00


In [21]:
df_renamed["value"].describe()

count    137088.000000
mean          6.615227
std          34.112858
min           0.000000
25%           0.000000
50%           3.090000
75%          10.920000
max        3000.000000
Name: value, dtype: float64

## Comment regarding the correctness of the exercise
Summarizing a dataset in this way is not entirely correct because all groups of products (preserves, meat, oat and other products) were treated as comparable. Moreover, because the data is formatted as a time series, all dates were treated the same.

This is commonly known as comparing oranges to apples. These statistics should at least be performed separately for each of the groups, and ideally sub-divided by quarter of the year.

In [38]:
# change types of the df_raw columns
df_typed = df_raw.astype(
    {
        "Name": "string",
        "Goods types": "string",
        "Measurement unit": "string",
        "Group ID": "Int64",
        "Product types": "string",
        "Value": "float",
    }
)

# replace missing values in Value column with 0
df_typed["Value"] = df_typed["Value"].fillna(0)

# jednoduché rozdělení (pokud všechny hodnoty mají formát "YYYY-M" nebo "YYYY-MM")
df_typed[["Year", "Month"]] = (
    df_typed["Date"]
    .astype(str)
    .str.split("-", expand=True)
    .iloc[:, :2]
    .apply(pd.to_numeric, errors="coerce")
    .astype("Int64")  # nullable integer dtype
)

# Delete column Date
df_typed = df_typed.drop(columns=["Date"])

# Create new column with quarter information
df_typed["Quarter"] = ((df_typed["Month"] - 1) // 3 + 1).astype("Int64")

# df_typed.head()  # display the first few rows and check the changes

In [37]:
df_typed

,Name,Goods types,Measurement unit,Group ID,Product types,Value,Year,Month,Quarter
0,SUBCARPATHIA,<NA>,PLN,2,pork ham cooked - per 1kg,21.37,2013,3,1
1,ŁÓDŹ,<NA>,PLN,4,bread - per 1kg,0.00,2018,2,1
2,KUYAVIA-POMERANIA,<NA>,PLN,2,barley groats sausage - per 1kg,3.55,2019,12,4
3,LOWER SILESIA,<NA>,PLN,2,dressed chickens - per 1kg,6.14,2019,2,1
4,WARMIA-MASURIA,<NA>,PLN,2,Italian head cheese - per 1kg,5.63,2002,3,1
...,...,...,...,...,...,...,...,...,...
149935,KUYAVIA-POMERANIA,<NA>,PLN,2,pork meat (raw bacon) - per 1kg,12.15,2016,11,4
149936,ŁÓDŹ,"beet sugar white, bagged - per 1kg",PLN,3,<NA>,0.00,2012,5,2
149937,LESSER POLAND,<NA>,PLN,4,plain mixed bread (wheat-rye) - per 1kg,3.05,2008,6,2
149938,WARMIA-MASURIA,<NA>,PLN,2,boneless beef (sirloin) - per 1kg,11.87,2000,11,4


---
**From:** 03_Day 1/01_Filtering/01_Filtering data.ipynb
---


In [85]:
data_path = Path.cwd() / "01_Data" / "product_prices_renamed.csv"
df = pd.read_csv(data_path, sep=";", encoding="UTF-8", decimal=",")

# change types of the df_raw columns
df_changed = df.astype(
    {
        "province": "string",
        "product_types": "string",
        "currency": "string",
        "product_line": "string",
        "value": "float",
        "date": "string",
    }
)

# change column date (string: 2018-1) to variable datetime (for example datetime: 2018-01-01)
date_split = df_changed["date"].str.split("-", expand=True)
years = pd.to_numeric(date_split[0], errors="coerce").astype("Int64")
months = pd.to_numeric(date_split[1], errors="coerce").astype("Int64")
df_changed["date"] = pd.to_datetime(
    dict(year=years, month=months, day=1), errors="coerce"
)

# ...existing code...
pt = df_changed["product_types"].astype("string").str.strip().replace("", pd.NA)
pl = df_changed["product_line"].astype("string").str.strip().replace("", pd.NA)
df_changed["product"] = pt.fillna(pl)
df_changed = df_changed.drop(columns=["product_types", "product_line"])
df_changed

# pokud chceš odstranit původní sloupce:
# df_changed = df_changed.drop(columns=["product_types", "product_line"])

# rychlá kontrola
# df_changed[["product_types", "product_line", "product"]].head()


# # delete rows with missing values in Value
# df_typed = df_typed.dropna(subset=["Value"])

# # Create new column with quarter information
# df_typed["Quarter"] = ((df_typed["Month"] - 1) // 3 + 1).astype("Int64")

,province,currency,product_group_id,value,date,product
0,SUBCARPATHIA,PLN,2,21.37,2013-03-01,pork ham cooked - per 1kg
1,ŁÓDŹ,PLN,4,NaN,2018-02-01,bread - per 1kg
2,KUYAVIA-POMERANIA,PLN,2,3.55,2019-12-01,barley groats sausage - per 1kg
3,LOWER SILESIA,PLN,2,6.14,2019-02-01,dressed chickens - per 1kg
4,WARMIA-MASURIA,PLN,2,5.63,2002-03-01,Italian head cheese - per 1kg
...,...,...,...,...,...,...
149935,KUYAVIA-POMERANIA,PLN,2,12.15,2016-11-01,pork meat (raw bacon) - per 1kg
149936,ŁÓDŹ,PLN,3,0.00,2012-05-01,"beet sugar white, bagged - per 1kg"
149937,LESSER POLAND,PLN,4,3.05,2008-06-01,plain mixed bread (wheat-rye) - per 1kg
149938,WARMIA-MASURIA,PLN,2,11.87,2000-11-01,boneless beef (sirloin) - per 1kg


In [71]:
print(df_changed["product_types"].unique())
print()
print(df_changed["product_line"].unique())

<StringArray>
[                                     <NA>,
 'whole pickled cucumbers 0.9l - per 1pc.',
       'fresh chichen egges - per 666pcs.',
        '30% tomato concentrate - per 1kg',
     'frozen carrot and pea mix - per 1kg',
      'beet sugar white, bagged - per 1kg',
             'apple juice, boxed - per 1l',
       'white table salt bagged - per 1kg',
       'natural chocolate plain - per 1kg']
Length: 9, dtype: string

<StringArray>
[                       'pork ham cooked - per 1kg',
                                  'bread - per 1kg',
                  'barley groats sausage - per 1kg',
                       'dressed chickens - per 1kg',
                    'Italian head cheese - per 1kg',
                                               <NA>,
                      'pork belly cooked - per 1kg',
 'pork  with bone (center-cut pork chop) - per 1kg',
          'plain mixed bread (wheat-rye) - per 1kg',
             'Poznan wheat flour, bagged - per 1kg',
                 'fr

# Filtering data
While generating the `product_prices_renamed.csv` file the following processing errors occurred and they need to be found:
1. In the **date** column, data from 1888 appeared - '1888-0',
2. Check the **date** column for similar errors but with future values.
3. In the **value** column, too high value was introduced – find it and locate the row where it is (use `query`),
4. There was a spelling error in the **product_types** column for one of the products. Find it and the corresponding rows. How many such rows are there?

You do not need to assign the results to the variable: it is enough if you display them.

> Based on the solution of this task, we will later correct all errors in the data.

Hints:

Subsection 2:
1. There is only one such value.
2. Use `loc` or `query` with the condition `date > '2020-1'`.

Subsection 3:
1. There is only one such value.
1. Do the following:
a) use `describe()`, to view percentiles,
b) use `loc` or `query` to find erroneous entries,

Subsection 4:

You can do it in the following way:
a) use `unique()` method to find all available values,
b) use `loc` or `query` to find erroneous entries,
c) The number of rows can be checked with the `shape` method.

---
**From:** 03_Day 1/01_Filtering/02_Selecting data.ipynb
---


## Selection of data for analysis

Answer the following questions:

1. What was the average price of a carrot and pea mix per 1 kg in Poland in 2015?
2. What was the average price of apple juice in 2016-2018 in the Masovia province?
3. What was the average price of tomato paste in Lower Silesia Province in 2003-2015? Compared with other products does it seem reasonable to you?<br>

Suggest what can be done with values equal to 0. How does this affect the results of point 3?

> All the product groups needed for this task are in the **product_types** column.

---
**From:** 03_Day 1/02_Data processing part 1/01_Duplicates.ipynb
---


# Modification of column names

Read the file **product_prices.csv** into a DataFrame. Modify the names of columns so that they match those in the  **product_prices_renamed.csv** file. Complete two variants of the exercise:

1. Using the `rename` method,
1. Overwriting all column names using `columns`.

To unify the naming and reduce the risk of a **typo**, you can use the dictionary below:
```
{'Name': 'province',
 'Types of goods': 'product_types',
 'Measurement unit': 'currency',
 'id': 'id',
 'Types of products': 'product_line',
 'Value': 'value',
 'Date': 'date'}
```

> The `columns` method overwrites the columns directly on the object (does not return a new DataFrame), so if you do step 2 first, the hint will be useless (the columns get overwritten).

---
**From:** 03_Day 1/02_Data processing part 1/02_Highest and lowest price.ipynb
---


# Highest and lowest price

Using `sort_values` and the data from the _product_prices_renamed.csv_ the file answer the questions below — use data aggregated for entire Poland (`province=='POLAND'`):

1. What product had the highest 'PLN' price in history? (remember to remove the value = 3000)?
1. What product achieved the lowest 'PLN' price in history? (remember to filter out values = 0),

Sort the data according to the following key:

1. by **product_types** column – descending
1. by **value** column – ascending

---
**From:** 03_Day 1/02_Data processing part 1/03_Duplicates.ipynb
---


# Duplicates

Using the **product_prices_renamed.csv** files:

1. check how many rows are duplicated,
1. using `duplicated` check which rows were doubled.
1. using `drop_duplicates`, remove all duplicates from the output dataset (we assume that a row with a duplicate is an error and needs to be removed) - write the result to a new DataFrame.

### Hints:

#### Subsection 2:
To determine, the number of duplicate rows use the source data and the results from subsection 1.

#### Subsection 3:

1. first use `duplicated`, to find duplicated rows,
1. use `loc` to separate them from the output set,
1. use `drop_duplicates` to get only those rows that are duplicated.

---
**From:** 03_Day 1/03_Data processing part 2/01_Removing errors.ipynb
---


# Removing errors
Using the solutions from the previous class and the **product_prices_renamed.csv** file, modify `loc` so that it corrects the errors present in the dataset:

1. In the **date** column, data from 1888 appeared - '1888-0', change the value to 1999-1,
1. In the **date** column, data from 2099 appeared - '2099-13', change the value to 2019-1,
1. There is a spelling error in the **product_types** column - correct it. Number of pieces should be '10pcs.`. Check whether the task was done correctly.
1. Use `loc` to convert the values given in `EUR` to `PLN` with 4.15 exchange rate.
1. Filter from the set those rows where the price for the product is 3000.
Hint: Instead of writing `loc` twice, first query the data for rows where **currency** = `EUR` and save it to a variable.

> Remember that `loc` modifies data irrevocably.

---
**From:** 03_Day 1/03_Data processing part 2/02_Adding a column.ipynb
---


# Adding a column

Following on the solution of the previous exercise, create a new column: **product** using **product_types** and **product_line**:

1. check that the **product_types** and **product_line** columns are complementary (an empty value in one column entails a non-empty value in the other).
1. create a new column: **product** using the values from the **product_types** column e.g. `df['product'] = df['product_types']`.
1. find non-empty values in the **product_line** column and enter them into the **product** column,
1. use a method of your choice to check if all values in the **product** column are non-empty.
1. remove duplicates from the table.
1. using the `to_csv` method save the data (we are going to use it later in the course), set separator=';' and `index=False`.<br>
Save the file as `product_prices_cleaned.csv`

The `read_csv` method is one of many that can be used to save a `DataFrame` as a **csv** file. Within the scope of this exercise we are interested in the following parameters.
- `sep`- row separator (default ','),
- `index`- is the index (row number by default) of the table to be saved as well (default: `True`).

Sample call:
```
df.to_csv(
    'filepath',
    sep=';', # separator setting
    index=False
)
```

---
**From:** 03_Day 1/04_Grouping/01_Grouping products.ipynb
---


# Grouping products

Using the data from the  **product_prices_cleaned.csv** file for provinces, do the following:

1. What was the average monthly price of each commodity?
1. Which product had the highest price volatility over the years?

Use the **product** and **value** columns for analysis.

Additionally:

1. Consider whether any more assumptions are needed for the tasks.
1. Why can this task be done only now, after cleaning the data? Do you think there are any more operations needed?

---
**From:** 03_Day 1/04_Grouping/02_Aggregation.ipynb
---


# Aggregation


Using data from file **product_prices_cleaned.csv**, aggregate the data for each product by month and determine the statistics: `min, max, median, mean, std` for prices (**value** column):

1. skip the national data in the analysis,
1. directly on the object from `groupby`,
1. write a loop that will calculate these values for individual provinces.

Use the `agg` method, and aggregate the data with the `'product','date'` column to complete the exercise.

---
**From:** 04_Day 2/01_Merge/01_Dataset normalization.ipynb
---


# Dataset normalization

In the sheets of the **dims.xlsx** file there are dictionaries for the data from the **product_prices_cleaned.csv** file. Use `merge` to normalize the data following the steps:

1. Read the contents of the **dims.xlsx** file sheets to separate `DataFrames`.
For readability base names of frames on the names of sheets.

1. Read the data from **product_prices_cleaned.csv** file to the `df` variable.

1. Based on the **d_province** workbook, use the `id` column to add the `province_id` column to the `df` frame.

1. Based on the  **d_product** workbook, add the `product_id` column to the `df` frame.

1. From the table, extract only the columns that refer to other tables, e.g.. **product_id** and the columns **value**, **date**. Do you think this is more readable? What are potential benefits of this approach?

> We will tell you how to read many workbooks at once when we discuss `openpyxl`.

You can find more about database normalization at the [link](https://www.sqlshack.com/what-is-database-normalization-in-sql-server/).

---
**From:** 04_Day 2/01_Merge/02_Merge with conditions.ipynb
---


# Merge with conditions

Using the raw data from the **product_prices_cleaned.csv** file, perform the task of finding out how many times a lower price has been quoted for a given product, province, month historically. To do this, perform the following steps:

1. merge the table with itself. What type of join should be used?
1. filter the data to find earlier years and the values smaller than the one from current year in a given province.
1. group the data accordingly.

Which product(s) had the most such occurrences?

> See what happens to the column names when the columns are not used as a merge condition, but have the same names

---
**From:** 04_Day 2/02_Datetime/01_Formatting dates.ipynb
---


# Formatting dates

Load the _product_prices_cleaned.csv_ file, the _date_ column contains information about the months of value reporting.

1. What date format is it?
1. Change the column format to `datetime64`.
1. Add a new column – _month_ to the DataFrame as a way to isolate the information about the month from the _date_ column.
1. Add a new column – _quarter_ to the DataFrame as a way to isolate the information about the quarter from the _date_ column,
1. Add a new column – _year_ to the DataFrame as a way to isolate the information about the year from the _date_ column,
1. Using the `dt.strfime` method ([link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.strftime.html) to the documentation), convert the  _date_ column to the _YYYY-MM-01_ format and overwrite its value; e.g.:
```
df['date'] = col.dt.strftime('format')
```
1. Overwrite the _product_prices_cleaned.csv_ file. It will be used in the following sections to analyze the collection.


---
**From:** 04_Day 2/03_Pivot/01_Pivot table.ipynb
---


# Pivot table

Using the data from the  **product_prices_cleaned.csv** file do the following:

1. create a pivot table so that the rows hold years, and columns have names of goods. Use the average price of the product as the value,
1. using `lambda` and `pivot_table`, recreate the view of the data discussed during the lecture. Analyse the result of the recreation, what can you say about what was passed to the function?
1. using `pivot_table`, see how the mean and median prices in the product groups were shaped across years (many functions can be passed as a list).

Subsection 2 hint:

Code from the lecture:
```
pd.pivot(data=df, index=['province', 'product'], columns=['date'], values=['value'])
```

---
**From:** 04_Day 2/04_Openpxl/01_File conversion.ipynb
---


# File conversion

Using `openpyxl` convert the **product_prices_cleaned.csv** file to an Excel file.

---
**From:** 04_Day 2/04_Openpxl/02_Grouping data across sheets.ipynb
---


# Grouping data across sheets

Modify the previous exercise so that each product group is in a separate worksheet together with the corresponding `product_group_id`. For example
`product_group_id` 1 should go to the sheet named `1`.

Use the method to create a new workbook:
```
wb.create_sheet(name, index)
```
Where:
- `name` - is the name of the workbook, where we want to save the data (should be a string),
- `index` - is the worksheet position in the workbook.

---
**From:** 04_Day 2/04_Openpxl/03_Automation for analysts.ipynb
---


# Automation for the analyst

A team of analysts prepares the monthly report on the prices of the product selected by the Board. Because they are aware you know Python, they asked you to automate the process. Talking to the team, you have set the following business conditions that enable process automation:

Three report parameters are available:
- **product_group_id**,
- **product**,
- **date**.

Assumptions for each parameter:

1. A parameter may have at most one value,
1. If the parameter is empty we return all records from the group,
1. We assume that the file is always prepared correctly (we want to practice report automation, not error handling).

Based on the above requirements:

1. load the  **config.xlsx** file using `openpyxl`,
1. prepare appropriate conditions to filter data from **product_cleaned.csv**,
1. based on the conditions filter the frame,
1. aggregate the data using a **pivot_table**:
   a) index-product, province,
   b) columns-dates,
   c) value-average product price,
   d) remember to remove 0,
6. save the file to the spreadsheet any way you want.

Hints:

1. You can save individual filtering conditions to variables and then use them all to filter `DataFrame`, the same as writing them all as before i.e. `df.loc[var1 & var2]`
1. If you decide to write with Pandas, be careful with the parameters passed to the function (what happens if you set `index=False`?). Link to the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html).